In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
mode='prod'
#mode='test'
sample=0.01

https://nbviewer.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-A3-hugging_face_transformers.ipynb

In [ ]:
! ls /kaggle/input/ktrain-library/ktrain

!pip install facenet-pytorch --no-index --find-links=file:///kaggle/input/facenet_pytorch/ 

In [ ]:
!pip install ktrain --no-index --find-links=file:///kaggle/input/ktrain-library/ktrain

In [ ]:
#!pip install ktrain

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import tensorflow as tf

In [ ]:
import seaborn as sns

In [ ]:
import ktrain

In [ ]:
from ktrain import text

Load training dataset

In [ ]:
#df=pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
#df=pd.read_csv("/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/train.csv")
#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/test_labels.csv

#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/test.csv

df=pd.read_csv("../input/jigsaw-toxic-coomens-annotations/jigsaw_train_hate_annotationprob.csv",index_col=0)
if mode=='test':
    df=df.sample(frac=sample)

In [ ]:
df.head()

Load evaluation dataset

In [ ]:
df_valid=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv",index_col=0)
if mode=='test':
    df_valid=df_valid.sample(frac=sample)

Load comments to score

In [ ]:
comments_to_score=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv",index_col=0)
if mode=='test':
    comments_to_score=comments_to_score.sample(frac=sample)

Main Corpus

scoring

In [ ]:
w1,w2,w3,w4,w5,w6=[0.32, 1.5, 0.16, 1.5, 0.64, 1.5]
#w1,w2,w3,w4,w5,w6=[0.48841081, 0.21194874, 0.39584097, 0.40030452, 0.46899059, 0.42384744]

In [ ]:
df['score']=w1*df['toxic']+w2*df['severe_toxic']+w3*df['obscene']+w4*df['threat']+w5*df['insult']+w6*df['identity_hate']

Adjusting scores

In [ ]:
def adjust_score(x):
    if x>0:
        return x+2
    else:
        return x

In [ ]:
#df['score']=df['score']+1-df['neutral_speech_agreement']
df['score']=df['score']+1-df['neutral_speech_agreement']+df['offensive_agreement_rating']+df['hate_speech_agreement']

In [ ]:
df.rename(columns={'text':'sentences'},inplace=True)
#df.rename(columns={'sentiments_tags':'labels'},inplace=True)

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")
ax = sns.displot(x="score", data=df)

Train test split

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.25)
#test, valid = train_test_split(test, test_size=0.05)

In [ ]:
#train
x_train =train.sentences.values
y_train = train.score.values
x_test = test.sentences.values
y_test = test.score.values
#validation
X_less_toxic =df_valid.less_toxic.values
X_more_toxic =df_valid.more_toxic.values
#submit file
X_comments_to_score =comments_to_score.text.values

Create a Text Regression Model and Wrap in Learner

Type of methods available
fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
linreg: linear text regression using a trainable Embedding layer
bigru: Bidirectional GRU with pretrained English word vectors [https://arxiv.org/abs/1712.09405]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) - keras_bert implementation [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]

Prepare the vector . Internet is not available so there is a need for an workaround. 

In [ ]:
!mkdir /root/ktrain_data

In [ ]:
cp -v ../input/fasttext-pretrainedvectors-english-text/cc.en.300.vec  /root/ktrain_data/

Bigru

In [ ]:
trn, val, preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          ngram_range=1, 
                                          maxlen=300, 
                                          max_features=50000)

In [ ]:
model = text.text_regression_model('bigru', train_data=trn, preproc=preproc)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

In [ ]:
learner.lr_find()

In [ ]:
learner.lr_plot()

In [ ]:
#learner.autofit(4e-4, 8)
learner.autofit(5e-5, 10)

In [ ]:
learner.plot('loss')

In [ ]:
learner.fit_onecycle(4e-4, 4)

In [ ]:
learner.plot('loss')

Check the validation score

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
p1 = predictor.predict(X_less_toxic)
p2 = predictor.predict(X_more_toxic)
# Validation Accuracy
(p1 < p2).mean()

Now let's see Roberta in a regression task. Maybe a 0.01 gain in validation

In [ ]:
#MODEL_NAME = 'distilbert-base-uncased'
#MODEL_NAME = 'roberta-base'
MODEL_NAME = '../input/roberta-base/'
t = text.Transformer(MODEL_NAME, maxlen=180)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)


In [ ]:
model = t.get_regression_model()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

In [ ]:
learner.fit_onecycle(4e-5, 2)

In [ ]:
learner.plot('loss')

In [ ]:
predictor2 = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
predictor2.batch_size = 196

In [ ]:
p3 = predictor2.predict(X_less_toxic)
p4 = predictor2.predict(X_more_toxic)
# Validation Accuracy
(p3 < p4).mean()

Undersampling majority class

Samplig for testing purpose
-to be commenteted for submission

In [ ]:
comments_to_score.head()

In [ ]:
#df['score']=df['toxic']+2*df['severe_toxic']+3*df['obscene']+4*df['threat']+5*df['identity_hate']

In [ ]:
#df['score']=2*df['score']

**STEP 5: Making Predictions**

Let's make a prediction for a random text in the validation set.

Using bert for regression

Using the Transfomer API for Text Regression
ktrain includes a simplified interface to the Hugging Face transformers library. This interface can also be used for text regression. Here is a short example of training a BERT model for a single epoch to predict text likes.

Using the Transfomer API for Text Regression

Evaluation using validation_data.csv

Verificare interna.

In [ ]:
from scipy.stats import rankdata

In [ ]:
predicted_scores1 = predictor.predict(X_comments_to_score)
predicted_scores2 = predictor2.predict(X_comments_to_score)

In [ ]:
comments_to_score['score1']=predicted_scores1
comments_to_score['score2']=predicted_scores2

In [ ]:
comments_to_score['rank1']=rankdata( comments_to_score['score1'], method='ordinal')
comments_to_score['rank2']=rankdata( comments_to_score['score2'], method='ordinal')
comments_to_score['blended_rank']=(comments_to_score['rank1']+comments_to_score['rank2'])/2
comments_to_score['score']=rankdata( comments_to_score['blended_rank'], method='ordinal')

Testare

In [ ]:
#comments_to_score['score']=predicted_scores

In [ ]:
#submission=comments_to_score[['score']]
#submission=comments_to_score[['blended_rank']]
# submission=comments_to_score[['rank2']]
#comments_to_score['score']=comments_to_score['score2']
submission=comments_to_score[['score']]

In [ ]:
comments_to_score.head()

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
submission.head()